In [272]:
import numpy as np
import matplotlib.pyplot as plt

In [273]:
with open('../headlinesData/Bad.csv') as bad:
    bad = bad.read().split('\n')
    

In [274]:
bad

['"FTC Sues Block $ 8.5 Billion Deal Combine Makers Coach , Michael Kors"',
 'Verizon Stock Falls After Revenue Misses Expectations',
 'Insurer Earnings Make Hospital Investors Sweat',
 '� Made Advertising � Websites Are Marketing Industry � Latest Messy Situation',
 'PwC Looks Shrink New U.S . Consulting Partner Class More Than 50 %',
 '"Companies Belly Up Cash Buffet , Five Charts"',
 'U.S . Fines Thai Company SCG Plastics $ 20 Million Over Alleged Iran Sanctions Violations',
 '"Real-Estate Agents , Investment Advisers Chafe New Anti-Money-Laundering Rules"',
 'Prologis Warns Slowing Industrial Real-Estate Market',
 '"CSX Revenue , Profit Fall Despite Rising Freight Volumes"',
 'Salesforce Calls AI Emissions Regulations Concerns Grow Over Tech Sector � Carbon Footprint',
 '"FTC Blocks Tapestry , Capri $ 8.5 Billion Merger ."',
 'GE Aerospace � Earnings Are Going Be Strange . Stay Tuned .',
 'Tesla Post Earnings After Tough Start Year',
 'Cathie Wood � Popular ARK Funds Are Sinking Fa

In [275]:
with open('../headlinesData/Good.csv') as good:
    good = good.read().split('\n')

In [276]:
with open('../headlinesData/Neutral.csv') as neutral:
    neutral = neutral.read().split('\n')

In [277]:
import sklearn as sk
from sklearn.feature_extraction.text import TfidfVectorizer

v = TfidfVectorizer()

good = v.fit_transform(good)

bad = v.transform(bad)

neutral = v.transform(neutral)

In [278]:
from sklearn import svm

clf = svm.SVC(kernel='linear')

In [279]:
trainGood = good[:int(good.shape[0]*.8)]
trainBad = bad[:int(bad.shape[0]*.8)]
trainNeutral = neutral[:int(neutral.shape[0]*.8)]

testGood = good[int(good.shape[0]*.8):]
testBad = bad[int(bad.shape[0]*.8):]
testNeutral = neutral[int(neutral.shape[0]*.8):]

In [280]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='hinge', penalty='l2', max_iter=1000, tol=1e-3)
def train():
    goodStep = int(trainGood.shape[0]/10)
    badStep = int(trainBad.shape[0]/10)
    neutralStep = int(trainNeutral.shape[0]/10)
    for i in range(9):
        goodSlice = trainGood[i*goodStep:(i+1)*goodStep].toarray()
        badSlice = trainBad[i*badStep:(i+1)*badStep].toarray()
        neutralSlice = trainNeutral[i*neutralStep:(i+1)*neutralStep].toarray()

        slices = [s for s in [goodSlice, badSlice, neutralSlice] if s.shape[0] > 0]
        if not slices:
            continue

        train_data = np.concatenate(slices)
        train_labels = np.concatenate((np.ones(goodSlice.shape[0]), np.zeros(badSlice.shape[0]), np.zeros(neutralSlice.shape[0])))
        clf.partial_fit(train_data, train_labels, classes=np.array([0, 1]))

In [281]:
train()

In [282]:
old_scores = []

In [283]:
def test():
    goodStep = int(testGood.shape[0]/10)
    badStep = int(testBad.shape[0]/10)
    neutralStep = int(testNeutral.shape[0]/10)
    for i in range(9):
        goodSlice = testGood[i*goodStep:(i+1)*goodStep].toarray()
        badSlice = testBad[i*badStep:(i+1)*badStep].toarray()
        neutralSlice = testNeutral[i*neutralStep:(i+1)*neutralStep].toarray()

        slices = [s for s in [goodSlice, badSlice, neutralSlice] if s.shape[0] > 0]
        if not slices:
            continue

        test_data = np.concatenate(slices)
        test_labels = np.concatenate((np.ones(goodSlice.shape[0]), np.zeros(badSlice.shape[0]), np.zeros(neutralSlice.shape[0])))
        print(clf.score(test_data, test_labels))
        old_scores.append(clf.score(test_data, test_labels))

In [284]:
test()

0.8571428571428571
0.7142857142857143
0.8571428571428571
0.5714285714285714
0.5714285714285714
0.5714285714285714
0.5714285714285714
0.5714285714285714
0.42857142857142855


In [285]:
#TODO: maybe try adding a column for what the stock rise or fall was the day before? that would take a lot of work though, as data would have to be split up and given a date. maybe make two models, this current one and one that combines the news direction with the stock direction

In [286]:
from nltk.tokenize import word_tokenize
from afinn import Afinn

In [287]:
afinn = Afinn()


In [288]:
def sentiment(textArray):
    return [afinn.score(text) for text in textArray]

In [289]:
with open('../headlinesData/Good.csv') as goodText:
    goodText = goodText.read().split('\n')
    
with open('../headlinesData/Bad.csv') as badText:
    badText = badText.read().split('\n')

with open('../headlinesData/Neutral.csv') as neutralText:
    neutralText = neutralText.read().split('\n')

In [290]:
goodSentiment = sentiment(goodText)
badSentiment = sentiment(badText)
neutralSentiment = sentiment(neutralText)

In [291]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

v = TfidfVectorizer()

good_df = pd.DataFrame({'text': goodText, 'sentiment': goodSentiment})
bad_df = pd.DataFrame({'text': badText, 'sentiment': badSentiment})
neutral_df = pd.DataFrame({'text': neutralText, 'sentiment': neutralSentiment})


In [292]:
good_tf = v.fit_transform(good_df['text'])
bad_tf = v.transform(bad_df['text'])
neutral_tf = v.transform(neutral_df['text'])

good_features = np.hstack([good_tf.toarray(), good_df['sentiment'].values.reshape(-1, 1)])
bad_features = np.hstack([bad_tf.toarray(), bad_df['sentiment'].values.reshape(-1, 1)])
neutral_features = np.hstack([neutral_tf.toarray(), neutral_df['sentiment'].values.reshape(-1, 1)])
    
    

In [293]:
good_train = good_features[:int(good_features.shape[0]*.8)]
bad_train = bad_features[:int(bad_features.shape[0]*.8)]
neutral_train = neutral_features[:int(neutral_features.shape[0]*.8)]

good_test = good_features[int(good_features.shape[0]*.8):]
bad_test = bad_features[int(bad_features.shape[0]*.8):]
neutral_test = neutral_features[int(neutral_features.shape[0]*.8):]

clf = SGDClassifier(loss='hinge', penalty='l2', max_iter=1000, tol=1e-3)

In [294]:
def reTrain():
    good_step = int(good_train.shape[0]/20)
    bad_step = int(bad_train.shape[0]/20)
    neutral_step = int(neutral_train.shape[0]/20)
    
    for i in range(19):
        good_slice = good_train[i*good_step:(i+1)*good_step]
        bad_slice = bad_train[i*bad_step:(i+1)*bad_step]
        neutral_slice = neutral_train[i*neutral_step:(i+1)*neutral_step]
        
        slices = [s for s in [good_slice, bad_slice, neutral_slice] if s.shape[0] > 0]
        if not slices:
            continue
        
        train_data = np.concatenate(slices)
        train_labels = np.concatenate((np.ones(good_slice.shape[0]), np.zeros(bad_slice.shape[0]), np.zeros(neutral_slice.shape[0])))
        clf.partial_fit(train_data, train_labels, classes=np.array([0, 1]))

In [295]:
reTrain()

In [296]:
new_scores =[]

In [297]:
def reTest():
    good_step = int(good_test.shape[0]/10)
    bad_step = int(bad_test.shape[0]/10)
    neutral_step = int(neutral_test.shape[0]/10)
    
    for i in range(9):
        good_slice = good_test[i*good_step:(i+1)*good_step]
        bad_slice = bad_test[i*bad_step:(i+1)*bad_step]
        neutral_slice = neutral_test[i*neutral_step:(i+1)*neutral_step]
        
        slices = [s for s in [good_slice, bad_slice, neutral_slice] if s.shape[0] > 0]
        if not slices:
            continue
        
        test_data = np.concatenate(slices)
        test_labels = np.concatenate((np.ones(good_slice.shape[0]), np.zeros(bad_slice.shape[0]), np.zeros(neutral_slice.shape[0])))
        new_scores.append(clf.score(test_data, test_labels))

In [298]:
reTest()

In [299]:
old = np.array(old_scores).mean();
new = np.array(new_scores).mean();

In [300]:
old

0.6349206349206349

In [301]:
new

0.6349206349206349